In [11]:
import numpy as np
import pandas as pd

import sys
import os

sys.path.append(os.path.abspath('..'))
from utils.db import get_db
from utils.transaction_generator import generate_transactions
from utils.basket_lift import BasketLift

In [13]:
engine = get_db()

In [14]:
query = """select sale_date, sale_id, product_code, product_description, qty, total_sales, sub_department_description,
sale_type, item_ring_type
from sales
where store_number='440'
and sale_date between '5/10/2025' and '5/15/2025'
and sale_type in ('Refunded', 'Sale')
and item_ring_type in ('ITEM')"""

In [15]:
ds = pd.read_sql(query,engine)

2025-06-12 12:58:29,216 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-12 12:58:29,217 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-12 12:58:29,271 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-12 12:58:29,271 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-12 12:58:29,325 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-12 12:58:29,326 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-12 12:58:29,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-12 12:58:29,384 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [16]:
# lets filter this down to just items
ds = ds[ds['item_ring_type'] == 'ITEM']

In [17]:
#now lets get rid of any empty products
ds.dropna(subset=['product_description', 'sale_id'], inplace=True)

In [18]:
# convert sale_id to string
ds['sale_id'] = ds['sale_id'].astype(str)

In [19]:
transactions = ds.groupby('sale_id')['product_description'].apply(list)

In [20]:
transactions.head()

sale_id
102481     [PREMIUM]
102482    [UNLEADED]
102483    [UNLEADED]
102484    [UNLEADED]
102486    [UNLEADED]
Name: product_description, dtype: object

In [21]:
filtered = transactions[transactions.apply(lambda x: len(x) > 1)]
transactions = filtered.tolist()

In [ ]:
bl = BasketLift(min_support=0.1, min_confidence=0.5, min_lift=1)
bl.fit(transactions)
bl.print_rules()